In [11]:
# DONT CHANGE
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import json
from selenium.webdriver.chrome.service import Service

driver = webdriver.Chrome(service=Service(r"C:\Users\30032339\Anaconda3\Lib\site-packages\chromedriver_autoinstaller\126\chromedriver.exe"))
# tr

url = 'https://www.upl-ltd.com/br/defensivos-agricolas'

driver.get(url)


def extract_product_details(products):
    """Extract product details from a list of products"""

    # list to store the products details
    product_list = []
    for product in products:
        try:
            try:
                # extract the product name
                name_element = product.find_element(
                    By.CSS_SELECTOR, 'div.product-name h3')

                product_name = name_element.text.strip()

                if not product_name:
                    continue

            except Exception as e:
                print(f"Error extracting product name: {e}")
                continue

            # Extract the product link
            try:
                link_element = product.find_element(
                    By.CSS_SELECTOR, 'a.view-btn')
                product_link = link_element.get_attribute('href')

            except Exception as e:
                print(f"Error extracting product link: {e}")
                continue

            try:
                driver.execute_script(
                    "window.open(arguments[0], '_blank');", product_link)
                driver.switch_to.window(driver.window_handles[1])

                WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located(
                        (By.CSS_SELECTOR, '.crop-row'))
                )

                try:
                    cultivos_clave_elements = driver.find_elements(
                        By.CSS_SELECTOR, '.crop-row')
                    cultivos_clave = [
                        driver.execute_script(
                            'return arguments[0].textContent;', elem).strip()
                        for elem in cultivos_clave_elements
                    ]

                except Exception as e:
                    print(f"error extracting cultivos_clave information for link {product_link}: {e}")
                    cultivos_clave = []

                driver.close()
                driver.switch_to.window(driver.window_handles[0])

                product = {
                    'product_name': product_name,
                    'product_link': product_link,
                    'cultivos_clave': cultivos_clave
                }

                product_list.append(product)

                print(f"{product_name}")

            except Exception as e:
                print(f"error handling product link {product_link}: {e}")
                driver.close()
                # switch back to main window
                driver.switch_to.window(driver.window_handles[0])
                continue

        except Exception as e:
            print(f'error processing a product: {e}')
            continue

    return product_list


try:
    # find all accordions
    accordion_headings = WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located(
            (By.CSS_SELECTOR, 'div.panel-heading.products-heading[data-toggle="collapse"]'))
    )

    all_data = []

    for accordion_heading in accordion_headings:
        try:
            # use js to click the accordion heading to expand it
            driver.execute_script("arguments[0].click();", accordion_heading)

            # extract the accordion text from spans inside the h2
            accordion_text = accordion_heading.find_element(
                By.CSS_SELECTOR, 'h2').text.strip()

            # wait for the products to be loaded
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located(
                    (By.CSS_SELECTOR, 'div.products'))
            )

            # show all the hidden products using js
        
            driver.execute_script("""
            var hiddenElements = document.querySelectorAll('.products-row.hidden');
            hiddenElements.forEach(function(elem) {
                elem.classList.remove('hidden');
                elem.style.display = 'block';
            });
        """)

            # find all product elements specific to the expanded accordion
            products = driver.find_elements(By.CSS_SELECTOR, f'div.product')

            # extract product details
            product_details = extract_product_details(products)

            # apppend the accordion and product details to the main list
            all_data.append({
                'accordion_text': accordion_text,
                'products': product_details
            })

        except Exception as e:
            print(f'Error processing accordion')
            continue

    # write the extracted data to json file
    with open('product_data.json', 'w', encoding='utf-8') as f:
        json.dump(all_data, f, ensure_ascii=False, indent=4)

    print("Data saved to json file")

except Exception as e:
    print(f'something went wrong {e}')

finally:
    driver.quit()


Data saved to json file


In [50]:
# from selenium import webdriver
# from selenium.webdriver.chrome.service import Service
# from selenium.webdriver.chrome.options import Options

# # # Set up Chrome options
# # options = Options()
# # options.add_argument('--lang=en')
# # prefs = {
# #     "translate_whitelists": {"pt":"en"},
# #     "translate":{"enabled":True}
# # }
# # options.add_experimental_option("prefs", prefs)

# # Initialize the Chrome driver
# driver = webdriver.Chrome(service=Service(r"C:\Users\30032339\Anaconda3\Lib\site-packages\chromedriver_autoinstaller\126\chromedriver.exe"), options=options)

# # Open the URL
# url = 'https://www.upl-ltd.com/br/defensivos-agricolas'
# driver.get(url)

# # Wait for the translation to complete (you might need to adjust the wait time)
# import time
# time.sleep(30)  # Wait for 5 seconds

# # Get the page source after translation
# translated_page_source = driver.page_source

# # Print the translated page source
# print(translated_page_source)

# # Close the driver
# driver.quit()


In [43]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import json
from selenium.webdriver.chrome.service import Service
from time import sleep
from selenium.webdriver.chrome.options import Options
import pandas as pd

# options = Options()
# prefs = {'intl.accept_languages': 'en,en_US'}

# options.add_experimental_option("prefs", prefs)
# driver = webdriver.Chrome(service=Service(r"C:\Users\30032339\Anaconda3\Lib\site-packages\chromedriver_autoinstaller\126\chromedriver.exe"),
#                            options=options)
driver = webdriver.Chrome(service=Service(r"C:\Users\30032339\Anaconda3\Lib\site-packages\chromedriver_autoinstaller\126\chromedriver.exe"))


url = 'https://www.upl-ltd.com/br/defensivos-agricolas'

driver.get(url)

# find all accordions
accordion_headings = WebDriverWait(driver, 30).until(
    EC.presence_of_all_elements_located(
        (By.CSS_SELECTOR, 'div.panel-heading.products-heading[data-toggle="collapse"]'))
)

all_data = []

for i,accordion_heading in enumerate(accordion_headings):
#     try:
    # use js to click the accordion heading to expand it
    driver.execute_script("arguments[0].click();", accordion_heading)
    
    ''' Fetches the category name and count from the accordion text'''
    # extract the accordion text from spans inside the h2
    accordion_text = accordion_heading.find_element(
        By.CSS_SELECTOR, 'h2').text.strip()
    print(accordion_text)
                
    '''
    * The WebDriverWait object repeatedly checks for the element matching the CSS selector 'div.products' within the specified timeout (10 seconds).
    * If the element is found and is present in the DOM (i.e., has been loaded on the page), 
        the until method returns the element itself, and your script continues execution.
    * If the element is not found within the 30-second timeout, a TimeoutException is raised, 
        and your script might need to handle this exception (usually by logging an error or retrying the action
    '''
    # wait for the products to be loaded
    WebDriverWait(driver, 30).until(
        EC.presence_of_element_located(
            (By.CSS_SELECTOR, 'div.products'))
    )
    '''Traverses a list of hidden product rows, 
    removes the hidden class (effectively unhides the rows) ,
    sets their display style to block, effectively making them visible on the webpage'''
    # show all the hidden products using js
    driver.execute_script("""
            var hiddenElements = document.querySelectorAll('.products-row.hidden');
            hiddenElements.forEach(function(elem) {
                elem.classList.remove('hidden');
                elem.style.display = 'block';
            });
        """)
    # find all product elements specific to the expanded accordion
    products = driver.find_elements(By.CSS_SELECTOR, 'div.product')
    j=1
    p_names=[]
    p_ings=[]
    p_links=[]
    for product in products:

        try:
            name=product.find_element(By.CSS_SELECTOR,'div.product-name h3')
            product_name=name.text.strip()

            if product_name not in ['',' ']:
#                     print(j)
#                     print(product_name)
                p_names.append(product_name)

            else:
               continue
#             except:
#                 break
            
               
        except Exception as e:
                print(f"Error extracting product name: {e}")
                continue
        try:    
            ingredients=[ing.text.strip().replace(' /','') for ing in product.find_elements(By.CSS_SELECTOR,'span.product-ingrd')]
#             print(ingredients)
            p_ings.append(ingredients)
        except Exception as e:
            print(f'Error fetching active ingredients list : {e}')
            continue
        try:
            link_element = product.find_element( By.CSS_SELECTOR, 'a.view-btn')
            product_link = link_element.get_attribute('href')
#             print(product_link)
            p_links.append(product_link)
            
        except Exception as e:
                print(f"Error extracting product link: {e}")
                continue
#         print('\n')
        j+=1
    temp_df=pd.DataFrame({'Category':accordion_text,'Product Name':p_names,'Active ingredients':p_ings,'Product link':p_links})
    display(temp_df)
    all_data.append(temp_df)
#     sleep(600)
      
driver.quit()
country_df=pd.concat(all_data)



HERBICIDAS (33)


,Category,Product Name,Active ingredients,Product link
0,HERBICIDAS (33),AGILE,"[Cletodim 240,00 (g/L), Haloxifope-P-metílico ...",https://www.upl-ltd.com/br/defensivos-agricola...
1,HERBICIDAS (33),ALBERO,"[FLUROXIPIR-MEPTÍLICO 115,0 g/L, Piccloram 80,...",https://www.upl-ltd.com/br/defensivos-agricola...
2,HERBICIDAS (33),ALIA,[sulfentrazona 500 (g/L) SC],https://www.upl-ltd.com/br/defensivos-agricola...
3,HERBICIDAS (33),ARTYS,"[picloram 64,00 (g/L) (equivalente ácido de Pi...",https://www.upl-ltd.com/br/defensivos-agricola...
4,HERBICIDAS (33),BROWSER,[picloram 240 (g/L) SL],https://www.upl-ltd.com/br/defensivos-agricola...
5,HERBICIDAS (33),CARDEAL,[fomesafem 250 (g/L) SL],https://www.upl-ltd.com/br/defensivos-agricola...
6,HERBICIDAS (33),CRESCENDO,[triclopir 480 (g/L) (equivalente em ácido de ...,https://www.upl-ltd.com/br/defensivos-agricola...
7,HERBICIDAS (33),DANADO,[picloram 240 (g/L) (equivalente ácido de Picl...,https://www.upl-ltd.com/br/defensivos-agricola...
8,HERBICIDAS (33),DINAMIC,[Amicarbazona 700 (g/Kg) WG],https://www.upl-ltd.com/br/defensivos-agricola...
9,HERBICIDAS (33),FAMOSO,"[picloram 64 (g/L), 2,4-D-dimetilamina 240 (g/...",https://www.upl-ltd.com/br/defensivos-agricola...


FUNGICIDAS (14)


,Category,Product Name,Active ingredients,Product link
0,FUNGICIDAS (14),AGILE,"[Cletodim 240,00 (g/L), Haloxifope-P-metílico ...",https://www.upl-ltd.com/br/defensivos-agricola...
1,FUNGICIDAS (14),ALBERO,"[FLUROXIPIR-MEPTÍLICO 115,0 g/L, Piccloram 80,...",https://www.upl-ltd.com/br/defensivos-agricola...
2,FUNGICIDAS (14),EMINENT GOLD,[Tetraconazol 230 ME],https://www.upl-ltd.com/br/defensivos-agricola...
3,FUNGICIDAS (14),EVOLUTION,"[Mancozeb 525 g/Kg, Azoxistrobina 37,5 g/Kg, P...",https://www.upl-ltd.com/br/defensivos-agricola...
4,FUNGICIDAS (14),GALAPUS SB,[Clorotalonil 720 g/L],https://www.upl-ltd.com/br/defensivos-agricola...
5,FUNGICIDAS (14),KASUMIN,"[Casugamicina 20 g/L, Monoetilenoglicol 50 g/L]",https://www.upl-ltd.com/br/defensivos-agricola...
6,FUNGICIDAS (14),MANZATE WG,[Mancozeb 750 g/Kg],https://www.upl-ltd.com/br/defensivos-agricola...
7,FUNGICIDAS (14),MICROTHIOL DISPERSS WG,[Enxofre 800 WG],https://www.upl-ltd.com/br/defensivos-agricola...
8,FUNGICIDAS (14),ORTHOCIDE 500,[Captana 500 k/kg],https://www.upl-ltd.com/br/defensivos-agricola...
9,FUNGICIDAS (14),PROPLANT,[Cloridrato de Propamocarbe 722 g/L],https://www.upl-ltd.com/br/defensivos-agricola...


REGULADOR DE CRESCIMENTO DE PLANTAS (3)


...

,Category,Product Name,Active ingredients,Product link
0,FISIOATIVADORES (3),BIOZYME,[],https://www.upl-ltd.com/br/defensivos-agricola...
1,FISIOATIVADORES (3),K-TIONIC,[],https://www.upl-ltd.com/br/defensivos-agricola...
2,FISIOATIVADORES (3),RAIZAL,[],https://www.upl-ltd.com/br/defensivos-agricola...


ACARICIDA (2)


,Category,Product Name,Active ingredients,Product link
0,ACARICIDA (2),OBNY,[Ciflumetofem 200 g/l],https://www.upl-ltd.com/br/defensivos-agricola...
1,ACARICIDA (2),OMITE 720 EC,[Propargito 720 g/L],https://www.upl-ltd.com/br/defensivos-agricola...


INSETICIDA MICROBIOLÓGICO (2)


,Category,Product Name,Active ingredients,Product link
0,INSETICIDA MICROBIOLÓGICO (2),TACKLER,"[Beauveria bassiana 100,0 g/L]",https://www.upl-ltd.com/br/defensivos-agricola...
1,INSETICIDA MICROBIOLÓGICO (2),ZEBU,[Metarhizium anisopliae 50 g/kg],https://www.upl-ltd.com/br/defensivos-agricola...


In [44]:
country_df

,Category,Product Name,Active ingredients,Product link
0,HERBICIDAS (33),AGILE,"[Cletodim 240,00 (g/L), Haloxifope-P-metílico ...",https://www.upl-ltd.com/br/defensivos-agricola...
1,HERBICIDAS (33),ALBERO,"[FLUROXIPIR-MEPTÍLICO 115,0 g/L, Piccloram 80,...",https://www.upl-ltd.com/br/defensivos-agricola...
2,HERBICIDAS (33),ALIA,[sulfentrazona 500 (g/L) SC],https://www.upl-ltd.com/br/defensivos-agricola...
3,HERBICIDAS (33),ARTYS,"[picloram 64,00 (g/L) (equivalente ácido de Pi...",https://www.upl-ltd.com/br/defensivos-agricola...
4,HERBICIDAS (33),BROWSER,[picloram 240 (g/L) SL],https://www.upl-ltd.com/br/defensivos-agricola...
...,...,...,...,...
2,FISIOATIVADORES (3),RAIZAL,[],https://www.upl-ltd.com/br/defensivos-agricola...
0,ACARICIDA (2),OBNY,[Ciflumetofem 200 g/l],https://www.upl-ltd.com/br/defensivos-agricola...
1,ACARICIDA (2),OMITE 720 EC,[Propargito 720 g/L],https://www.upl-ltd.com/br/defensivos-agricola...
0,INSETICIDA MICROBIOLÓGICO (2),TACKLER,"[Beauveria bassiana 100,0 g/L]",https://www.upl-ltd.com/br/defensivos-agricola...


In [1]:
def extract_product_details(products):
    """Extract product details from a list of products"""

    # list to store the products details
    product_list = []
    for product in products:
        try:
            try:
                # extract the product name
                name_element = product.find_element(
                    By.CSS_SELECTOR, 'div.product-name h3')

                product_name = name_element.text.strip()

                if not product_name:
                    continue

            except Exception as e:
                print(f"Error extracting product name: {e}")
                continue

            # Extract the product link
            try:
                link_element = product.find_element(
                    By.CSS_SELECTOR, 'a.view-btn')
                product_link = link_element.get_attribute('href')

            except Exception as e:
                print(f"Error extracting product link: {e}")
                continue

            try:
                driver.execute_script(
                    "window.open(arguments[0], '_blank');", product_link)
                driver.switch_to.window(driver.window_handles[1])

                WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located(
                        (By.CSS_SELECTOR, '.crop-row'))
                )

                try:
                    cultivos_clave_elements = driver.find_elements(
                        By.CSS_SELECTOR, '.crop-row')
                    cultivos_clave = [
                        driver.execute_script(
                            'return arguments[0].textContent;', elem).strip()
                        for elem in cultivos_clave_elements
                    ]

                except Exception as e:
                    print(f"error extracting cultivos_clave information for link {product_link}: {e}")
                    cultivos_clave = []

                driver.close()
                driver.switch_to.window(driver.window_handles[0])

                product = {
                    'product_name': product_name,
                    'product_link': product_link,
                    'cultivos_clave': cultivos_clave
                }

                product_list.append(product)

                print(f"{product_name}")

            except Exception as e:
                print(f"error handling product link {product_link}: {e}")
                driver.close()
                # switch back to main window
                driver.switch_to.window(driver.window_handles[0])
                continue

        except Exception as e:
            print(f'error processing a product: {e}')
            continue

    return product_list